In [ ]:
import torch
from torch import nn

In [ ]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [ ]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7 이 되도록 avg pooling 하는 녀석
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    # mode는 분산을 조절해주는 기능을 한다.
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)   # 미분값이 0이 아님. 헷갈리지 마라.
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg: # cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
            if type(v) == int:
              # vggnet 이후에 batch_norm이 추가된 것.
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1, bias=False), # 어차피 BN에 bias 포함
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)]
        return nn.Sequential(*layers)

In [ ]:
# (4,4)가 되도록 알아서 맞춰주는 것이다.
# output size로 딱 떨어지지 않는 경우. 계산이 어렵기 때문에 그런 경우 이 함수가 매우 유용해진다.
avgpool = nn.AdaptiveAvgPool2d((4, 4))
# print(avgpool(torch.randn(2,3,32,32)).shape)
print(avgpool(torch.randn(2,3,2,2))) # 작은 놈이 들어오면 늘려서라도 맞춰준다 # 값을 복제 시켜놓음

tensor([[[[-0.1377, -0.1377,  2.1122,  2.1122],
          [-0.1377, -0.1377,  2.1122,  2.1122],
          [-0.4069, -0.4069,  0.1511,  0.1511],
          [-0.4069, -0.4069,  0.1511,  0.1511]],

         [[-0.2866, -0.2866, -1.0811, -1.0811],
          [-0.2866, -0.2866, -1.0811, -1.0811],
          [-1.8162, -1.8162, -0.0108, -0.0108],
          [-1.8162, -1.8162, -0.0108, -0.0108]],

         [[-0.4842, -0.4842,  0.7408,  0.7408],
          [-0.4842, -0.4842,  0.7408,  0.7408],
          [ 0.3263,  0.3263,  1.4850,  1.4850],
          [ 0.3263,  0.3263,  1.4850,  1.4850]]],


        [[[-0.0209, -0.0209, -0.9257, -0.9257],
          [-0.0209, -0.0209, -0.9257, -0.9257],
          [ 0.1507,  0.1507, -2.0558, -2.0558],
          [ 0.1507,  0.1507, -2.0558, -2.0558]],

         [[ 0.4016,  0.4016,  0.7454,  0.7454],
          [ 0.4016,  0.4016,  0.7454,  0.7454],
          [ 0.1527,  0.1527,  0.1296,  0.1296],
          [ 0.1527,  0.1527,  0.1296,  0.1296]],

         [[-2.0951, -2.0951,

In [ ]:
model = nn.Sequential(nn.Linear(2,2),
                      nn.ReLU(),
                      nn.Sequential(nn.Linear(2,3),
                                    nn.ReLU(),
                                    nn.Linear(3,3),
                                    nn.ReLU()),
                      nn.Linear(3,10))
[m for m in model.modules()]

[Sequential(
   (0): Linear(in_features=2, out_features=2, bias=True)
   (1): ReLU()
   (2): Sequential(
     (0): Linear(in_features=2, out_features=3, bias=True)
     (1): ReLU()
     (2): Linear(in_features=3, out_features=3, bias=True)
     (3): ReLU()
   )
   (3): Linear(in_features=3, out_features=10, bias=True)
 ),
 Linear(in_features=2, out_features=2, bias=True),
 ReLU(),
 Sequential(
   (0): Linear(in_features=2, out_features=3, bias=True)
   (1): ReLU()
   (2): Linear(in_features=3, out_features=3, bias=True)
   (3): ReLU()
 ),
 Linear(in_features=2, out_features=3, bias=True),
 ReLU(),
 Linear(in_features=3, out_features=3, bias=True),
 ReLU(),
 Linear(in_features=3, out_features=10, bias=True)]

In [ ]:
# model1 = nn.Sequential([nn.Linear(1,1),
#                        nn.Linear(1,1)]) # 리스트를 넣으면 안돼요!

# model2 = nn.Sequential(nn.Linear(1,1),
#                        nn.Linear(1,1))

# print(*[1,2])

model3 = nn.Sequential(*[nn.Linear(1,1),
                         nn.Linear(1,1)])

In [ ]:
model = VGG(cfgs["E"], batch_norm=False)
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [2, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [2, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [2, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [2, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [2, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [2, 256, 56, 56]          29

In [ ]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


In [ ]:
x = torch.randn(2,3,300,300)
print(model(x).shape)

torch.Size([2, 1000])


In [ ]:
x = torch.randn(2,3,32,32)
print(model(x).shape)

torch.Size([2, 1000])
